In [235]:
# import pandas and geopandas
import pandas as pd
import geopandas as gpd

# needed to get information on the zip code
import pgeocode


import folium
from folium.plugins import HeatMap, MarkerCluster
from folium import GeoJson

In [236]:
# Read geopandas dataframe
gdf = gpd.read_file("../data/usa-hospital-beds.geojson")
#geodf.head()
# Read regular dataframe
df = pd.read_csv("../data/usa-hospital-beds.csv")
#df.head()

In [239]:
# We verify if there are any NA values in our data
df.isnull().sum()

X                                    18
Y                                    18
OBJECTID                              0
HOSPITAL_NAME                         0
HOSPITAL_TYPE                         0
HQ_ADDRESS                            0
HQ_ADDRESS1                        6252
HQ_CITY                               0
HQ_STATE                              0
HQ_ZIP_CODE                           0
COUNTY_NAME                          25
STATE_NAME                           25
STATE_FIPS                           25
CNTY_FIPS                            25
FIPS                                 25
NUM_LICENSED_BEDS                   153
NUM_STAFFED_BEDS                    241
NUM_ICU_BEDS                          0
ADULT_ICU_BEDS                        0
PEDI_ICU_BEDS                       737
BED_UTILIZATION                     714
Potential_Increase_In_Bed_Capac       0
AVG_VENTILATOR_USAGE                  0
dtype: int64

In [241]:
# Keeping where X exists
df = df[df['X'].notna()]

In [242]:
# Checking again
df.isnull().sum()

X                                     0
Y                                     0
OBJECTID                              0
HOSPITAL_NAME                         0
HOSPITAL_TYPE                         0
HQ_ADDRESS                            0
HQ_ADDRESS1                        6237
HQ_CITY                               0
HQ_STATE                              0
HQ_ZIP_CODE                           0
COUNTY_NAME                           7
STATE_NAME                            7
STATE_FIPS                            7
CNTY_FIPS                             7
FIPS                                  7
NUM_LICENSED_BEDS                   150
NUM_STAFFED_BEDS                    238
NUM_ICU_BEDS                          0
ADULT_ICU_BEDS                        0
PEDI_ICU_BEDS                       732
BED_UTILIZATION                     709
Potential_Increase_In_Bed_Capac       0
AVG_VENTILATOR_USAGE                  0
dtype: int64

In [243]:
features = ["OBJECTID","X","Y"]
coord_df = df[features]

In [244]:
gdf = gdf.merge(coord_df, on="OBJECTID")
#geodf.head(15)

In [246]:
gdf.head(-20)

,OBJECTID,HOSPITAL_NAME,HOSPITAL_TYPE,HQ_ADDRESS,HQ_ADDRESS1,HQ_CITY,HQ_STATE,HQ_ZIP_CODE,COUNTY_NAME,STATE_NAME,...,NUM_STAFFED_BEDS,NUM_ICU_BEDS,ADULT_ICU_BEDS,PEDI_ICU_BEDS,BED_UTILIZATION,Potential_Increase_In_Bed_Capac,AVG_VENTILATOR_USAGE,geometry,X,Y
0,1,Phoenix VA Health Care System (AKA Carl T Hayd...,VA Hospital,650 E Indian School Rd,None,Phoenix,AZ,85012,Maricopa,Arizona,...,62.0,0,0,NaN,NaN,0,0.0,POINT (-112.06616 33.49550),-112.066157,33.495498
1,2,Southern Arizona VA Health Care System,VA Hospital,3601 S 6th Ave,None,Tucson,AZ,85723,Pima,Arizona,...,295.0,2,2,NaN,NaN,0,2.0,POINT (-110.96589 32.18126),-110.965885,32.181263
2,3,VA Central California Health Care System,VA Hospital,2615 E Clinton Ave,None,Fresno,CA,93703,Fresno,California,...,54.0,2,2,NaN,NaN,0,2.0,POINT (-119.77974 36.77332),-119.779742,36.773323
3,4,VA Connecticut Healthcare System - West Haven ...,VA Hospital,950 Campbell Ave,None,West Haven,CT,6516,New Haven,Connecticut,...,216.0,1,1,NaN,NaN,0,2.0,POINT (-72.95761 41.28440),-72.957610,41.284400
4,5,Wilmington VA Medical Center,VA Hospital,1601 Kirkwood Hwy,None,Wilmington,DE,19805,New Castle,Delaware,...,62.0,0,0,NaN,NaN,0,1.0,POINT (-75.60653 39.74021),-75.606533,39.740206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6578,6597,PFC James Dunn VA Clinic (AKA Pueblo VA Clinic),Short Term Acute Care Hospital,4776 Eagleridge Cir,None,Pueblo,CO,81008,Pueblo,Colorado,...,NaN,0,0,NaN,NaN,0,0.0,POINT (-104.61936 38.32295),-104.619358,38.322948
6579,6598,Joint Base Cape Cod Field Hospital (Temporaril...,Short Term Acute Care Hospital,Connery Ave,None,Buzzards Bay,MA,2542,Barnstable,Massachusetts,...,NaN,0,0,NaN,NaN,0,0.0,POINT (-70.54954 41.65419),-70.549535,41.654188
6580,6599,UMass Lowell Recreation Center Field Hospital ...,Short Term Acute Care Hospital,322 Aiken St,None,Lowell,MA,1854,Middlesex,Massachusetts,...,NaN,0,0,NaN,NaN,0,0.0,POINT (-71.31706 42.65178),-71.317060,42.651779
6581,6600,Suburban Collection Showplace Field Hospital (...,Short Term Acute Care Hospital,46100 Grand River Ave,None,Novi,MI,48374,Oakland,Michigan,...,NaN,0,0,NaN,NaN,0,0.0,POINT (-83.50269 42.48914),-83.502688,42.489142


In [247]:
gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [248]:
# And we also project the overall gdf to the same EPSG 
proj_gdf = gdf.to_crs(epsg=2163)
proj_gdf

,OBJECTID,HOSPITAL_NAME,HOSPITAL_TYPE,HQ_ADDRESS,HQ_ADDRESS1,HQ_CITY,HQ_STATE,HQ_ZIP_CODE,COUNTY_NAME,STATE_NAME,...,NUM_STAFFED_BEDS,NUM_ICU_BEDS,ADULT_ICU_BEDS,PEDI_ICU_BEDS,BED_UTILIZATION,Potential_Increase_In_Bed_Capac,AVG_VENTILATOR_USAGE,geometry,X,Y
0,1,Phoenix VA Health Care System (AKA Carl T Hayd...,VA Hospital,650 E Indian School Rd,None,Phoenix,AZ,85012,Maricopa,Arizona,...,62.0,0,0,NaN,NaN,0,0.0,POINT (-1119937.868 -1197620.307),-112.066157,33.495498
1,2,Southern Arizona VA Health Care System,VA Hospital,3601 S 6th Ave,None,Tucson,AZ,85723,Pima,Arizona,...,295.0,2,2,NaN,NaN,0,2.0,POINT (-1035048.033 -1356105.197),-110.965885,32.181263
2,3,VA Central California Health Care System,VA Hospital,2615 E Clinton Ave,None,Fresno,CA,93703,Fresno,California,...,54.0,2,2,NaN,NaN,0,2.0,POINT (-1746146.288 -706485.100),-119.779742,36.773323
3,4,VA Connecticut Healthcare System - West Haven ...,VA Hospital,950 Campbell Ave,None,West Haven,CT,6516,New Haven,Connecticut,...,216.0,1,1,NaN,NaN,0,2.0,POINT (2210123.741 -43418.017),-72.957610,41.284400
4,5,Wilmington VA Medical Center,VA Hospital,1601 Kirkwood Hwy,None,Wilmington,DE,19805,New Castle,Delaware,...,62.0,0,0,NaN,NaN,0,1.0,POINT (2050505.132 -278500.067),-75.606533,39.740206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6598,6617,Ascension Seton Bastrop,Short Term Acute Care Hospital,630 Highway 71 W,None,Bastrop,TX,78602,Bastrop,Texas,...,NaN,0,0,NaN,NaN,0,0.0,POINT (263744.352 -1647828.340),-97.280937,30.101578
6599,6618,Tower Behavioral Health,Psychiatric Hospital,201 Wellness Way,None,Reading,PA,19605,Berks,Pennsylvania,...,NaN,0,0,NaN,NaN,0,0.0,POINT (2001398.407 -219136.887),-75.972565,40.374573
6600,6619,Mid Jefferson Extended Care Hospital - Beaumont,Short Term Acute Care Hospital,860 S 8th St,None,Beaumont,TX,77701,Jefferson,Texas,...,NaN,0,0,NaN,NaN,0,0.0,POINT (569630.003 -1636194.153),-94.125229,30.069448
6601,6620,Stat Specialty Hospital - Laredo North,Short Term Acute Care Hospital,2502 Ne Bob Bullock Loop,None,Laredo,TX,78045,Webb,Texas,...,NaN,0,0,NaN,NaN,0,0.0,POINT (52865.624 -1925721.990),-99.469612,27.613489


In [249]:
proj_gdf.crs

<Projected CRS: EPSG:2163>
Name: US National Atlas Equal Area
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: USA
- bounds: (167.65, 15.56, -65.69, 74.71)
Coordinate Operation:
- name: US National Atlas Equal Area
- method: Lambert Azimuthal Equal Area (Spherical)
Datum: Not specified (based on Clarke 1866 Authalic Sphere)
- Ellipsoid: Clarke 1866 Authalic Sphere
- Prime Meridian: Greenwich

In [250]:
country_init = pgeocode.Nominatim("us")

In [251]:
#zip_query = country_init.query_postal_code(form.zip_code.data)
zip_data = 85012
zip_query = country_init.query_postal_code(zip_data)
zip_query

postal_code          85012
country code            US
place_name         Phoenix
state_name         Arizona
state_code              AZ
county_name       Maricopa
county_code             13
community_name         NaN
community_code         NaN
latitude           33.5097
longitude         -112.068
accuracy                 4
Name: 0, dtype: object

In [252]:
zip_df = pd.DataFrame(
    {"City":[zip_query.place_name], "State": [zip_query.state_name], 
     "X": [zip_query.longitude], "Y": [zip_query.latitude]})
zip_df

,City,State,X,Y
0,Phoenix,Arizona,-112.0678,33.5097


In [253]:
# We make a geopandas dataframe with the query data
geozip = gpd.GeoDataFrame(zip_df,crs=4326, geometry=gpd.points_from_xy(zip_df.X, zip_df.Y))
geozip

,City,State,X,Y,geometry
0,Phoenix,Arizona,-112.0678,33.5097,POINT (-112.06780 33.50970)


In [254]:
# Then we project the geometry to the US National Atlas Equal Area (uses meters)
proj_geozip = geozip.to_crs(epsg=2163)
proj_geozip

,City,State,X,Y,geometry
0,Phoenix,Arizona,-112.0678,33.5097,POINT (-1119890.675 -1196036.236)


In [255]:
proj_geozip.crs

<Projected CRS: EPSG:2163>
Name: US National Atlas Equal Area
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: USA
- bounds: (167.65, 15.56, -65.69, 74.71)
Coordinate Operation:
- name: US National Atlas Equal Area
- method: Lambert Azimuthal Equal Area (Spherical)
Datum: Not specified (based on Clarke 1866 Authalic Sphere)
- Ellipsoid: Clarke 1866 Authalic Sphere
- Prime Meridian: Greenwich

In [256]:
proj_gdf.crs

<Projected CRS: EPSG:2163>
Name: US National Atlas Equal Area
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: USA
- bounds: (167.65, 15.56, -65.69, 74.71)
Coordinate Operation:
- name: US National Atlas Equal Area
- method: Lambert Azimuthal Equal Area (Spherical)
Datum: Not specified (based on Clarke 1866 Authalic Sphere)
- Ellipsoid: Clarke 1866 Authalic Sphere
- Prime Meridian: Greenwich

In [257]:
distances = proj_gdf.geometry.distance(proj_geozip.iloc[0].geometry)
distances

0       1.584773e+03
1       1.811639e+05
2       7.948940e+05
3       3.523851e+06
4       3.300497e+06
            ...     
6598    1.455528e+06
6599    3.270593e+06
6600    1.745915e+06
6601    1.381231e+06
6602    4.075482e+05
Length: 6603, dtype: float64

In [263]:
km_distances = distances/1000
km_distances

0          1.584773
1        181.163866
2        794.893960
3       3523.850844
4       3300.497265
           ...     
6598    1455.528080
6599    3270.592893
6600    1745.914979
6601    1381.230841
6602     407.548165
Length: 6603, dtype: float64

In [267]:
print('Mean distance to hospitals: {} kilometers'.format(km_distances.mean()))

Mean distance to hospitals: 2142.222459378824 kilometers


In [268]:
print('Closest monitoring station ({} kilometers):'.format(km_distances.min()))

Closest monitoring station (1.19632881416161 kilometers):


In [282]:
best_match = proj_gdf.iloc[distances.idxmin()][["HQ_ADDRESS", "HQ_CITY","HQ_STATE" , "HQ_ZIP_CODE", "Y", "X"]]
best_match

HQ_ADDRESS     720 E Montebello Ave
HQ_CITY                     Phoenix
HQ_STATE                         AZ
HQ_ZIP_CODE                   85014
Y                           33.5202
X                          -112.065
Name: 5991, dtype: object

In [285]:
best_match.X

-112.0647645

In [278]:
m = folium.Map(location=[proj_geozip.iloc[0].Y, proj_geozip.iloc[0].X], zoom_start=11)
m

In [286]:
folium.Marker([proj_geozip.iloc[0].Y, proj_geozip.iloc[0].X],popup="You are here").add_to(m)

folium.Marker([best_match.Y, best_match.X],popup=best_match.HQ_ADDRESS).add_to(m)

m
